In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from groq import Groq
import json


client = Groq(api_key = os.getenv('GROQ_API_KEY'))

In [3]:
model = "llama-3.1-70b-versatile"

In [4]:
# Example dummy function hard coded to return the score of an NBA game
def get_game_score(team_name):
    """Get the current score for a given NBA game"""
    if "warriors" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "lakers" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "nuggets" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    elif "heat" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    else:
        return json.dumps({"team_name": team_name, "score": "unknown"})

In [5]:
def run_conversation(user_prompt):
    # Step 1: send the conversation and available functions to the model
    messages=[
        {
            "role": "system",
            "content": """You are a function calling LLM that uses the data extracted from the get_game_score function to answer questions around NBA game scores.
            Include the team and their opponent in your response."""
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_game_score",
                "description": "Get the score for a given NBA game",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "team_name": {
                            "type": "string",
                            "description": "The name of the NBA team (e.g. 'Golden State Warriors')",
                        }
                    },
                    "required": ["team_name"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    print(f"Initial Response: {response_message} \n")
    tool_calls = response_message.tool_calls

    print(f"{tool_calls} \n")
    # Step 2: check if the model wanted to call a function
    print("Final Response:")
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_game_score": get_game_score,
        }  # only one function in this example, but you can have multiple

        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                team_name=function_args.get("team_name")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=model,
            messages=messages
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content

    else:
        return response.choices[0].message.content

In [6]:
user_prompt = "What was the score of the Warriors game?"
print(run_conversation(user_prompt))

Initial Response: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5jvj', function=Function(arguments='{"team_name":"Golden State Warriors"}', name='get_game_score'), type='function')]) 

[ChatCompletionMessageToolCall(id='call_5jvj', function=Function(arguments='{"team_name":"Golden State Warriors"}', name='get_game_score'), type='function')] 

Final Response:
The Warriors' score was 128 against their opponents, the Los Angeles Lakers. The final score of the game was Warriors 128, Lakers 121.


In [7]:
print(run_conversation("What was the score of the Lakers?"))

Initial Response: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ff7h', function=Function(arguments='{"team_name":"Los Angeles Lakers"}', name='get_game_score'), type='function')]) 

[ChatCompletionMessageToolCall(id='call_ff7h', function=Function(arguments='{"team_name":"Los Angeles Lakers"}', name='get_game_score'), type='function')] 

Final Response:
The Lakers played the Golden State Warriors and the Lakers scored 121 points while the Warriors scored 128.


### Parallel Tool Usage

In [8]:
from tavily import TavilyClient
import json
tavily_client = TavilyClient()

response = tavily_client.qna_search("What is the weather in london now",max_results=1)

In [9]:
print(response)



The current weather in London is 28.6°C with patchy rain nearby. The humidity is at 43%, and the wind speed is 6.8 kph coming from ENE.


In [26]:
# Tool functions
def get_weather(location, date):
    """Get weather forecast for a location on a specific date"""
    # This is a mock function. In a real scenario, you'd call a weather API.
    weather_data = {
        "New York": {"temp": 22, "condition": "Sunny"},
        "London": {"temp": 18, "condition": "Cloudy"},
        "Tokyo": {"temp": 28, "condition": "Rainy"},
        "Paris": {"temp": 20, "condition": "Partly cloudy"}
    }
    if not date:
        date = "today"
    if location in weather_data:
        response = tavily_client.qna_search(f"What is/was the weather of {location} on {date}")
        return json.dumps(response)
    return json.dumps({"error": "Location not found"})

def get_flights(origin, destination, date):
    """Get available flights between two cities on a specific date"""
    # This is a mock function with more realistic flight data
    flight_routes = {
        ("New York", "Paris"): [
            {"airline": "Air France", "departure": "10:00", "arrival": "23:30", "price": 750},
            {"airline": "Delta", "departure": "14:30", "arrival": "04:00", "price": 820},
            {"airline": "United", "departure": "19:45", "arrival": "09:15", "price": 680}
        ],
        ("New York", "London"): [
            {"airline": "British Airways", "departure": "21:00", "arrival": "09:30", "price": 650},
            {"airline": "American Airlines", "departure": "18:15", "arrival": "06:45", "price": 700},
            {"airline": "Virgin Atlantic", "departure": "23:00", "arrival": "11:30", "price": 620}
        ],
        ("London", "Tokyo"): [
            {"airline": "Japan Airlines", "departure": "09:30", "arrival": "06:45", "price": 1200},
            {"airline": "ANA", "departure": "12:00", "arrival": "09:15", "price": 1150},
            {"airline": "British Airways", "departure": "19:00", "arrival": "16:30", "price": 1300}
        ],
        ("Paris", "New York"): [
            {"airline": "Air France", "departure": "11:30", "arrival": "14:00", "price": 780},
            {"airline": "Delta", "departure": "15:45", "arrival": "18:15", "price": 850},
            {"airline": "United", "departure": "20:30", "arrival": "23:00", "price": 710}
        ]
    }

    route = (origin, destination)
    if route in flight_routes:
        return json.dumps(flight_routes[route])
    elif (destination, origin) in flight_routes:
        # If the reverse route exists, we'll return it with a note
        reverse_flights = flight_routes[(destination, origin)]
        return json.dumps({
            "note": f"No direct flights found from {origin} to {destination}. Here are return flights:",
            "flights": reverse_flights
        })
    else:
        return json.dumps({"error": f"No flights found between {origin} and {destination}"})

def get_hotels(location, check_in, check_out):
    """Get available hotels in a location for specific dates"""
    # This is a mock function. In a real scenario, you'd call a hotel booking API.
    hotels = {
        "New York": [
            {"name": "The Ritz-Carlton New York", "rating": 5, "price_per_night": 500},
            {"name": "Hilton Midtown", "rating": 4, "price_per_night": 300},
            {"name": "Holiday Inn Manhattan", "rating": 3, "price_per_night": 150}
        ],
        "Paris": [
            {"name": "Four Seasons Hotel George V", "rating": 5, "price_per_night": 1000},
            {"name": "Mercure Paris Centre Tour Eiffel", "rating": 4, "price_per_night": 250},
            {"name": "Ibis Paris Eiffel Tower", "rating": 3, "price_per_night": 120}
        ],
        "London": [
            {"name": "The Savoy", "rating": 5, "price_per_night": 700},
            {"name": "Doubletree by Hilton London", "rating": 4, "price_per_night": 200},
            {"name": "Premier Inn London City", "rating": 3, "price_per_night": 100}
        ],
        "Tokyo": [
            {"name": "Park Hyatt Tokyo", "rating": 5, "price_per_night": 600},
            {"name": "Hotel Metropolitan Tokyo", "rating": 4, "price_per_night": 250},
            {"name": "APA Hotel Shinjuku", "rating": 3, "price_per_night": 80}
        ]
    }
    if location in hotels:
        return json.dumps(hotels[location])
    return json.dumps({"error": "No hotels found for the specified location"})

def get_attractions(location):
    """Get top attractions in a location"""
    # This is a mock function. In a real scenario, you'd call a tourism API.
    attractions = {
        "New York": ["Statue of Liberty", "Central Park", "Times Square", "Metropolitan Museum of Art"],
        "London": ["Big Ben", "Tower Bridge", "British Museum", "Buckingham Palace"],
        "Tokyo": ["Tokyo Skytree", "Senso-ji Temple", "Shibuya Crossing", "Meiji Shrine"],
        "Paris": ["Eiffel Tower", "Louvre Museum", "Notre-Dame Cathedral", "Arc de Triomphe"]
    }
    if location in attractions:
        return json.dumps(attractions[location])
    return json.dumps({"error": "Location not found"})

def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": """You are a helpful travel planning assistant.
            Use the provided tools to help users plan their trips, including checking weather, finding flights and hotels, and suggesting attractions."""
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get weather forecast for a location on a specific date, show the live weather data if date is not mentioned",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                        "date": {"type": "string", "description": "Date in YYYY-MM-DD format if mentioned, otherwise assume it as today"}
                    },
                    "required": ["location", "date"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_flights",
                "description": "Get available flights between two cities on a specific date",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure city"},
                        "destination": {"type": "string", "description": "Arrival city"},
                        "date": {"type": "string", "description": "Date in YYYY-MM-DD format"}
                    },
                    "required": ["origin", "destination", "date"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_hotels",
                "description": "Get available hotels in a location for specific dates",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                        "check_in": {"type": "string", "description": "Check-in date in YYYY-MM-DD format"},
                        "check_out": {"type": "string", "description": "Check-out date in YYYY-MM-DD format"}
                    },
                    "required": ["location", "check_in", "check_out"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_attractions",
                "description": "Get top attractions in a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                    },
                    "required": ["location"],
                },
            },
        },
    ]

    while True:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_tokens=4096
        )

        response_message = response.choices[0].message
        messages.append(response_message)

        if not response_message.tool_calls:
            return response_message.content

        available_functions = {
            "get_weather": get_weather,
            "get_flights": get_flights,
            "get_hotels": get_hotels,
            "get_attractions": get_attractions,
        }

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )



In [27]:
print(run_conversation("paris weather"))

The weather in Paris on July 31, 2024, is currently 33.9°C with thundery outbreaks in nearby areas. The humidity is at 42%, and there is a 30% cloud cover. The wind is coming from the south at 5.0 km/h. The visibility is at 9.0 km, and the UV index is 7.0.


In [33]:
# Example usage
user_prompt = "I want to plan a trip from New York to Paris for next week. Can you help me with the weather, flights, hotels, and attractions?"
print(run_conversation(user_prompt))

Based on the information you provided, it appears that the weather in Paris on March 17, 2024, will be light rain and drizzle with a temperature of around 68°F. 

There are several flight options available from New York to Paris, with prices ranging from $680 to $820. The flights are operated by Air France, Delta, and United, with departure times ranging from 10:00 AM to 19:45 PM.

For accommodations, there are several options available in Paris, ranging from luxury hotels like the Four Seasons Hotel George V to more budget-friendly options like the Ibis Paris Eiffel Tower. The prices per night range from $120 to $1000.

As for attractions, some of the top recommendations for Paris include the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and the Arc de Triomphe.

It seems like you have a good starting point for planning your trip to Paris. Would you like to explore any of these options further or do you have any other questions?


## Adding more complications

In [37]:
import os
from groq import Groq
import json
from datetime import datetime, timedelta
import random

# # Initialize the Groq client
# client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
# MODEL = 'llama3-groq-70b-8192-tool-use-preview'

# Enhanced tool functions
def get_weather(location, date):
    """Get weather forecast for a location on a specific date"""
    weather_data = {
        "New York": {"temp": 22, "condition": "Sunny", "humidity": 60, "wind_speed": 10},
        "London": {"temp": 18, "condition": "Cloudy", "humidity": 75, "wind_speed": 15},
        "Tokyo": {"temp": 28, "condition": "Rainy", "humidity": 80, "wind_speed": 8},
        "Paris": {"temp": 20, "condition": "Partly cloudy", "humidity": 65, "wind_speed": 12}
    }
    if date is None:
        date="now"
    if location in weather_data:
        response = tavily_client.qna_search(f"what is the weather of {location} on {date}")
        return json.dumps(response)
    return json.dumps({"error": "Location not found"})

def get_flights(origin, destination, date):
    """Get available flights between two cities on a specific date"""
    flight_routes = {
        ("New York", "Paris"): [
            {"airline": "Air France", "departure": "10:00", "arrival": "23:30", "price": 750, "stops": 0},
            {"airline": "Delta", "departure": "14:30", "arrival": "04:00", "price": 820, "stops": 1},
            {"airline": "United", "departure": "19:45", "arrival": "09:15", "price": 680, "stops": 0}
        ],
        ("New York", "London"): [
            {"airline": "British Airways", "departure": "21:00", "arrival": "09:30", "price": 650, "stops": 0},
            {"airline": "American Airlines", "departure": "18:15", "arrival": "06:45", "price": 700, "stops": 1},
            {"airline": "Virgin Atlantic", "departure": "23:00", "arrival": "11:30", "price": 620, "stops": 0}
        ],
        ("London", "Tokyo"): [
            {"airline": "Japan Airlines", "departure": "09:30", "arrival": "06:45", "price": 1200, "stops": 1},
            {"airline": "ANA", "departure": "12:00", "arrival": "09:15", "price": 1150, "stops": 0},
            {"airline": "British Airways", "departure": "19:00", "arrival": "16:30", "price": 1300, "stops": 1}
        ],
        ("Paris", "New York"): [
            {"airline": "Air France", "departure": "11:30", "arrival": "14:00", "price": 780, "stops": 0},
            {"airline": "Delta", "departure": "15:45", "arrival": "18:15", "price": 850, "stops": 1},
            {"airline": "United", "departure": "20:30", "arrival": "23:00", "price": 710, "stops": 0}
        ]
    }

    route = (origin, destination)
    if route in flight_routes:
        return json.dumps(flight_routes[route])
    elif (destination, origin) in flight_routes:
        reverse_flights = flight_routes[(destination, origin)]
        return json.dumps({
            "note": f"No direct flights found from {origin} to {destination}. Here are return flights:",
            "flights": reverse_flights
        })
    else:
        return json.dumps({"error": f"No flights found between {origin} and {destination}"})

def get_hotels(location, check_in, check_out):
    """Get available hotels in a location for specific dates"""
    hotels = {
        "New York": [
            {"name": "The Ritz-Carlton New York", "rating": 5, "price_per_night": 500, "amenities": ["spa", "pool", "gym"]},
            {"name": "Hilton Midtown", "rating": 4, "price_per_night": 300, "amenities": ["gym", "restaurant"]},
            {"name": "Holiday Inn Manhattan", "rating": 3, "price_per_night": 150, "amenities": ["wifi", "breakfast"]}
        ],
        "Paris": [
            {"name": "Four Seasons Hotel George V", "rating": 5, "price_per_night": 1000, "amenities": ["spa", "pool", "michelin-star restaurant"]},
            {"name": "Mercure Paris Centre Tour Eiffel", "rating": 4, "price_per_night": 250, "amenities": ["gym", "eiffel tower view"]},
            {"name": "Ibis Paris Eiffel Tower", "rating": 3, "price_per_night": 120, "amenities": ["wifi", "breakfast"]}
        ],
        "London": [
            {"name": "The Savoy", "rating": 5, "price_per_night": 700, "amenities": ["spa", "butler service", "river view"]},
            {"name": "Doubletree by Hilton London", "rating": 4, "price_per_night": 200, "amenities": ["gym", "restaurant"]},
            {"name": "Premier Inn London City", "rating": 3, "price_per_night": 100, "amenities": ["wifi", "family rooms"]}
        ],
        "Tokyo": [
            {"name": "Park Hyatt Tokyo", "rating": 5, "price_per_night": 600, "amenities": ["spa", "pool", "city view"]},
            {"name": "Hotel Metropolitan Tokyo", "rating": 4, "price_per_night": 250, "amenities": ["gym", "multiple restaurants"]},
            {"name": "APA Hotel Shinjuku", "rating": 3, "price_per_night": 80, "amenities": ["wifi", "public bath"]}
        ]
    }
    if location in hotels:
        return json.dumps(hotels[location])
    return json.dumps({"error": "No hotels found for the specified location"})

def get_attractions(location):
    """Get top attractions in a location"""
    attractions = {
        "New York": [
            {"name": "Statue of Liberty", "type": "Monument", "price": 25},
            {"name": "Central Park", "type": "Park", "price": 0},
            {"name": "Metropolitan Museum of Art", "type": "Museum", "price": 25},
            {"name": "Broadway Show", "type": "Entertainment", "price": 100}
        ],
        "London": [
            {"name": "Big Ben", "type": "Monument", "price": 0},
            {"name": "British Museum", "type": "Museum", "price": 0},
            {"name": "London Eye", "type": "Observation Wheel", "price": 30},
            {"name": "Tower of London", "type": "Historic Castle", "price": 25}
        ],
        "Tokyo": [
            {"name": "Tokyo Skytree", "type": "Observation Tower", "price": 20},
            {"name": "Senso-ji Temple", "type": "Temple", "price": 0},
            {"name": "Tsukiji Outer Market", "type": "Market", "price": 0},
            {"name": "TeamLab Borderless", "type": "Digital Art Museum", "price": 30}
        ],
        "Paris": [
            {"name": "Eiffel Tower", "type": "Monument", "price": 25},
            {"name": "Louvre Museum", "type": "Museum", "price": 15},
            {"name": "Notre-Dame Cathedral", "type": "Church", "price": 0},
            {"name": "Seine River Cruise", "type": "Tour", "price": 15}
        ]
    }
    if location in attractions:
        return json.dumps(attractions[location])
    return json.dumps({"error": "Location not found"})

def get_restaurants(location, cuisine=None):
    """Get restaurant recommendations in a location, optionally filtered by cuisine"""
    restaurants = {
        "New York": [
            {"name": "Le Bernardin", "cuisine": "French", "price_range": "$$$", "rating": 4.5},
            {"name": "Katz's Delicatessen", "cuisine": "American", "price_range": "$$", "rating": 4.2},
            {"name": "Joe's Pizza", "cuisine": "Italian", "price_range": "$", "rating": 4.3},
            {"name": "Morimoto", "cuisine": "Japanese", "price_range": "$$$", "rating": 4.4}
        ],
        "Paris": [
            {"name": "L'Ami Louis", "cuisine": "French", "price_range": "$$$", "rating": 4.3},
            {"name": "L'As du Fallafel", "cuisine": "Middle Eastern", "price_range": "$", "rating": 4.4},
            {"name": "Septime", "cuisine": "Modern French", "price_range": "$$$", "rating": 4.6},
            {"name": "Chez L'Ami Louis", "cuisine": "French", "price_range": "$$$", "rating": 4.2}
        ],
        "London": [
            {"name": "Dishoom", "cuisine": "Indian", "price_range": "$$", "rating": 4.5},
            {"name": "The Wolseley", "cuisine": "British", "price_range": "$$$", "rating": 4.3},
            {"name": "Padella", "cuisine": "Italian", "price_range": "$$", "rating": 4.4},
            {"name": "Sketch", "cuisine": "Modern European", "price_range": "$$$$", "rating": 4.2}
        ],
        "Tokyo": [
            {"name": "Sukiyabashi Jiro", "cuisine": "Japanese", "price_range": "$$$$", "rating": 4.7},
            {"name": "Ichiran", "cuisine": "Japanese", "price_range": "$$", "rating": 4.3},
            {"name": "Ginza Kyubey", "cuisine": "Japanese", "price_range": "$$$", "rating": 4.5},
            {"name": "Narisawa", "cuisine": "Innovative", "price_range": "$$$$", "rating": 4.6}
        ]
    }
    if location in restaurants:
        if cuisine:
            filtered_restaurants = [r for r in restaurants[location] if r['cuisine'].lower() == cuisine.lower()]
            return json.dumps(filtered_restaurants if filtered_restaurants else {"error": f"No {cuisine} restaurants found in {location}"})
        return json.dumps(restaurants[location])
    return json.dumps({"error": "Location not found"})

def get_transportation(location):
    """Get public transportation options in a location"""
    transportation = {
        "New York": [
            {"type": "Subway", "cost_per_ride": 2.75, "operates": "24/7"},
            {"type": "Bus", "cost_per_ride": 2.75, "operates": "24/7"},
            {"type": "Taxi", "base_fare": 2.50, "per_mile": 2.50},
            {"type": "Bike Share", "cost_per_30min": 3.50}
        ],
        "Paris": [
            {"type": "Metro", "cost_per_ride": 1.90, "operates": "5:30 AM - 1:15 AM"},
            {"type": "Bus", "cost_per_ride": 2.00, "operates": "5:30 AM - 12:30 AM"},
            {"type": "RER (Commuter Rail)", "cost_varies": True, "operates": "5:00 AM - 12:15 AM"},
            {"type": "Velib (Bike Share)", "cost_per_day": 5}
        ],
        "London": [
            {"type": "Underground (Tube)", "cost_varies": True, "operates": "5:00 AM - 12:00 AM"},
            {"type": "Bus", "cost_per_ride": 1.55, "operates": "24/7"},
            {"type": "Overground", "cost_varies": True, "operates": "5:00 AM - 12:00 AM"},
            {"type": "Santander Cycles", "cost_per_30min": 2}
        ],
        "Tokyo": [
            {"type": "Metro", "cost_varies": True, "operates": "5:00 AM - 1:00 AM"},
            {"type": "JR Trains", "cost_varies": True, "operates": "4:30 AM - 1:00 AM"},
            {"type": "Bus", "cost_per_ride": 210, "operates": "Varies by route"},
            {"type": "Taxi", "base_fare": 410, "per_km": 80}
        ]
    }
    if location in transportation:
        return json.dumps(transportation[location])
    return json.dumps({"error": "Location not found"})

def currency_converter(amount, from_currency, to_currency):
    """Convert currency based on latest exchange rates"""
    # This is a mock function. In a real scenario, you'd call a currency conversion API.
    exchange_rates = {
        "USD": 1.0,
        "EUR": 0.85,
        "GBP": 0.73,
        "JPY": 110.0
    }
    if from_currency in exchange_rates and to_currency in exchange_rates:
        result = amount * (exchange_rates[to_currency] / exchange_rates[from_currency])
        return json.dumps({"result": round(result, 2)})
    return json.dumps({"error": "Currency not supported"})

def get_events(location, start_date, end_date):
    """Get events in a location during a specific date range"""
    events = {
        "New York": [
            {"name": "Broadway Week", "date": "2024-09-03", "category": "Theater"},
            {"name": "New York Fashion Week", "date": "2024-09-10", "category": "Fashion"},
            {"name": "NYC Marathon", "date": "2024-11-03", "category": "Sports"}
        ],
        "Paris": [
            {"name": "Bastille Day", "date": "2024-07-14", "category": "National Holiday"},
            {"name": "Paris Autumn Festival", "date": "2024-09-10", "category": "Arts"},
            {"name": "Nuit Blanche", "date": "2024-10-05", "category": "Arts"}
        ],
        "London": [
            {"name": "Wimbledon", "date": "2024-07-01", "category": "Sports"},
            {"name": "Notting Hill Carnival", "date": "2024-08-25", "category": "Festival"},
            {"name": "London Film Festival", "date": "2024-10-07", "category": "Film"}
        ],
        "Tokyo": [
            {"name": "Sumida River Fireworks", "date": "2024-07-27", "category": "Festival"},
            {"name": "Tokyo Game Show", "date": "2024-09-12", "category": "Technology"},
            {"name": "Tokyo Marathon", "date": "2024-03-03", "category": "Sports"}
        ]
    }
    if location in events:
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        filtered_events = [
            event for event in events[location]
            if start <= datetime.strptime(event['date'], "%Y-%m-%d") <= end
        ]
        return json.dumps(filtered_events if filtered_events else {"message": f"No events found in {location} between {start_date} and {end_date}"})
    return json.dumps({"error": "Location not found"})

def get_travel_advisories(location):
    """Get travel advisories for a specific location"""
    advisories = {
        "New York": {"level": "Low", "description": "Exercise normal precautions"},
        "Paris": {"level": "Moderate", "description": "Exercise increased caution due to terrorism"},
        "London": {"level": "Low", "description": "Exercise normal precautions"},
        "Tokyo": {"level": "Low", "description": "Exercise normal precautions"}
    }
    if location in advisories:
        return json.dumps(advisories[location])
    return json.dumps({"error": "No travel advisory information available for this location"})

def get_language_phrases(location):
    """Get common phrases in the local language of a location"""
    phrases = {
        "New York": {"language": "English", "phrases": {
            "Hello": "Hello",
            "Thank you": "Thank you",
            "Goodbye": "Goodbye",
            "Where is...?": "Where is...?"
        }},
        "Paris": {"language": "French", "phrases": {
            "Hello": "Bonjour",
            "Thank you": "Merci",
            "Goodbye": "Au revoir",
            "Where is...?": "Où est...?"
        }},
        "London": {"language": "English", "phrases": {
            "Hello": "Hello",
            "Thank you": "Thank you",
            "Goodbye": "Cheers",
            "Where is...?": "Where's...?"
        }},
        "Tokyo": {"language": "Japanese", "phrases": {
            "Hello": "Konnichiwa",
            "Thank you": "Arigatou",
            "Goodbye": "Sayonara",
            "Where is...?": "... wa doko desu ka?"
        }}
    }
    if location in phrases:
        return json.dumps(phrases[location])
    return json.dumps({"error": "No language information available for this location"})

def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": "You are an advanced travel planning assistant. Use the provided tools to help users plan their trips, including checking weather, finding flights, hotels, restaurants, attractions, transportation options, currency conversion, events, travel advisories, and language assistance."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get weather forecast for a location on a specific date",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                        "date": {"type": "string", "description": "Date in YYYY-MM-DD format"}
                    },
                    "required": ["location", "date"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_flights",
                "description": "Get available flights between two cities on a specific date",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure city"},
                        "destination": {"type": "string", "description": "Arrival city"},
                        "date": {"type": "string", "description": "Date in YYYY-MM-DD format"}
                    },
                    "required": ["origin", "destination", "date"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_hotels",
                "description": "Get available hotels in a location for specific dates",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                        "check_in": {"type": "string", "description": "Check-in date in YYYY-MM-DD format"},
                        "check_out": {"type": "string", "description": "Check-out date in YYYY-MM-DD format"}
                    },
                    "required": ["location", "check_in", "check_out"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_attractions",
                "description": "Get top attractions in a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_restaurants",
                "description": "Get restaurant recommendations in a location, optionally filtered by cuisine",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                        "cuisine": {"type": "string", "description": "Type of cuisine (optional)"}
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_transportation",
                "description": "Get public transportation options in a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "currency_converter",
                "description": "Convert currency based on latest exchange rates",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number", "description": "Amount to convert"},
                        "from_currency": {"type": "string", "description": "Currency to convert from"},
                        "to_currency": {"type": "string", "description": "Currency to convert to"}
                    },
                    "required": ["amount", "from_currency", "to_currency"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_events",
                "description": "Get events in a location during a specific date range",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                        "start_date": {"type": "string", "description": "Start date in YYYY-MM-DD format"},
                        "end_date": {"type": "string", "description": "End date in YYYY-MM-DD format"}
                    },
                    "required": ["location", "start_date", "end_date"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_travel_advisories",
                "description": "Get travel advisories for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_language_phrases",
                "description": "Get common phrases in the local language of a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"},
                    },
                    "required": ["location"],
                },
            },
        },
    ]

    while True:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_tokens=4096
        )

        response_message = response.choices[0].message
        messages.append(response_message)

        if not response_message.tool_calls:
            return response_message.content

        available_functions = {
            "get_weather": get_weather,
            "get_flights": get_flights,
            "get_hotels": get_hotels,
            "get_attractions": get_attractions,
            "get_restaurants": get_restaurants,
            "get_transportation": get_transportation,
            "currency_converter": currency_converter,
            "get_events": get_events,
            "get_travel_advisories": get_travel_advisories,
            "get_language_phrases": get_language_phrases,
        }

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )



In [ ]:
# Example usage
user_prompt = """I want to plan a comprehensive trip from New York to Tokyo for next week.
Can you help me with all aspects of the trip, including weather, flights, hotels, attractions,
restaurants, transportation, currency exchange, events, travel advisories, and some basic French phrases?"""

print(run_conversation(user_prompt))

## LLama 3 function calling with langchain

In [48]:
del load_dotenv

NameError: name 'load_dotenv' is not defined

In [49]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [50]:
# from octoai.client import OctoAI
# from octoai.text_gen import ChatMessage

# client = OctoAI(api_key=os.getenv("OCTOAI_API_KEY"))
# for completion in client.text_gen.create_chat_completion_stream(
#     model="meta-llama-3-8b-instruct",
#     messages=[
#         ChatMessage(
#             role="system",
#             content="Below is an instruction that describes a task. Write a response that appropriately completes the request.",
#         ),
#         ChatMessage(role="user", content="Write a blog about Seattle"),
#     ],
#     max_tokens=150,
# ):
#     print(completion.choices[0].delta.content, end='', flush=True)

from langchain.chains import LLMChain
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
from langchain_core.prompts import PromptTemplate


template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n Instruction:\n{question}\n Response: """
prompt = PromptTemplate.from_template(template)

llm = OctoAIEndpoint(
    model_name="meta-llama-3.1-405b-instruct",
    max_tokens=512,
    presence_penalty=0,
    temperature=0.1,
    top_p=0.9,
)
question = "If new zealand is a country then what is zealand?"

chain = prompt | llm

print(chain.invoke(question))

In [84]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults
import webbrowser

from pprint import PrettyPrinter
printer = PrettyPrinter()

@tool
def web_search(query:str) -> str:
    """Search the web using tavily api"""
    client = TavilySearchResults()
    response = client.invoke({"query": query})
    return printer.pprint(response[1]['content'])


@tool
def open_webpage(url:str):
    """open the website when the user requests so"""
    webbrowser.open(url)

In [85]:
tools = [
    web_search,
    open_webpage
]

In [86]:
ChatPromptTemplate.from_messages

<bound method ChatPromptTemplate.from_messages of <class 'langchain_core.prompts.chat.ChatPromptTemplate'>>

In [101]:
## llm has been initialized before
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
assistant_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',
         'you are a helpful assistant with two tools assinged (1)web_search and (2)open_webpage'
         'You must use atleast one of the tools to answer the question, You must not assume and answer on your own'
         'You can use multiple tools'),
         ('placeholder',"{messages}"),
    ]
)

In [102]:
llm = ChatGroq(model="llama3-70b-8192",temperature=0.2)
assistant_runnable = assistant_prompt|llm.bind_tools(tools)

In [103]:
question = "what is zealandia?"
payload = assistant_runnable.invoke({"messages":[("user",question)]})
payload.tool_calls

[{'name': 'web_search',
  'args': {'query': 'what is zealandia'},
  'id': 'call_7r5t',
  'type': 'tool_call'}]

In [104]:
question = "open the website of delta airlines"
payload = assistant_runnable.invoke({"message":[("user", question)]})
payload.tool_calls

[{'name': 'web_search',
  'args': {'query': "what is the answer to the user's question"},
  'id': 'call_v3w2',
  'type': 'tool_call'}]

In [105]:
payload

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_v3w2', 'function': {'arguments': '{"query":"what is the answer to the user\'s question"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 1049, 'total_tokens': 1100, 'completion_time': 0.165701616, 'prompt_time': 0.108215145, 'queue_time': None, 'total_time': 0.273916761}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-49b183cc-4392-42c9-bc5f-7f7e02d7487e-0', tool_calls=[{'name': 'web_search', 'args': {'query': "what is the answer to the user's question"}, 'id': 'call_v3w2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1049, 'output_tokens': 51, 'total_tokens': 1100})